In [1]:
from backend.connect_to_api import ResRobot

resrobot = ResRobot()

# Departures from Gothenburg Central Station
gothenburg_id = 740000002
departure_data = resrobot.timetable_departure(gothenburg_id)
departure_data

2025-02-04 14:32:11.034 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-04 14:32:11.424 
  command:

    streamlit run c:\Users\lager\Documents\Github\travel_planner_2025\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-04 14:32:11.425 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


FileNotFoundError: No secrets found. Valid paths for a secrets.toml file or secret directories are: C:\Users\lager\.streamlit\secrets.toml, c:\Users\lager\Documents\Github\travel_planner_2025\explorations\.streamlit\secrets.toml

In [2]:
import pandas as pd

# Extract departure data
departures = departure_data.get("Departure", [])

df = pd.DataFrame(departures)

print(df.columns)
print(df.head())

Index(['JourneyDetailRef', 'JourneyStatus', 'ProductAtStop', 'Product',
       'Notes', 'name', 'type', 'stop', 'stopid', 'stopExtId', 'lon', 'lat',
       'time', 'date', 'reachable', 'direction', 'directionFlag', 'rtPlatform',
       'prognosisType', 'rtTime', 'rtDate', 'rtTrack'],
      dtype='object')
                   JourneyDetailRef JourneyStatus  \
0  {'ref': '1|112454|1|1|27012025'}             P   
1  {'ref': '1|180230|0|1|27012025'}             P   
2     {'ref': '1|356|0|1|27012025'}             P   
3     {'ref': '1|520|0|1|27012025'}             P   
4   {'ref': '1|22457|7|1|27012025'}             P   

                                       ProductAtStop  \
0  {'icon': {'res': 'prod_gen'}, 'operatorInfo': ...   
1  {'icon': {'res': 'prod_gen'}, 'operatorInfo': ...   
2  {'icon': {'res': 'prod_gen'}, 'operatorInfo': ...   
3  {'icon': {'res': 'prod_gen'}, 'operatorInfo': ...   
4  {'icon': {'res': 'prod_gen'}, 'operatorInfo': ...   

                                     

In [3]:
print(df["ProductAtStop"][0])

print(df[["time", "date", "direction"]].head())

{'icon': {'res': 'prod_gen'}, 'operatorInfo': {'name': 'Västtrafik', 'nameS': '279', 'nameN': '279', 'nameL': 'Västtrafik', 'id': '279'}, 'name': 'Länstrafik - Buss X3', 'internalName': 'Länstrafik - Buss X3', 'displayNumber': 'X3', 'num': 'X3', 'line': 'X3', 'lineId': '1279520300001', 'catOut': 'BLT', 'catIn': 'BLT', 'catCode': '7', 'cls': '128', 'catOutS': 'BLT', 'catOutL': 'Länstrafik - Buss', 'operatorCode': '279', 'operator': 'Västtrafik', 'admin': '279___', 'matchId': 'X3;5203;50'}
       time        date                       direction
0  08:46:00  2025-01-27               Göteborg Svingeln
1  08:46:00  2025-01-27                  Göteborg Heden
2  08:47:00  2025-01-27  Mölnlycke station (Härryda kn)
3  08:47:00  2025-01-27             Kungälv resecentrum
4  08:47:00  2025-01-27        Fyrktorget (Göteborg kn)


In [4]:
# Flatten ProductAtStop column
product_at_stop_df = pd.json_normalize(df["ProductAtStop"])

product_at_stop_df = product_at_stop_df[["name", "displayNumber", "catOutL"]]

# Combine the flattened data with the main DataFrame
df_combined = pd.concat([df, product_at_stop_df], axis=1)

print(
    df_combined[
        ["time", "date", "direction", "name", "displayNumber", "catOutL"]
    ].head()
)

       time        date                       direction  \
0  08:46:00  2025-01-27               Göteborg Svingeln   
1  08:46:00  2025-01-27                  Göteborg Heden   
2  08:47:00  2025-01-27  Mölnlycke station (Härryda kn)   
3  08:47:00  2025-01-27             Kungälv resecentrum   
4  08:47:00  2025-01-27        Fyrktorget (Göteborg kn)   

                    name                   name displayNumber  \
0   Länstrafik - Buss X3   Länstrafik - Buss X3            X3   
1  Länstrafik - Buss 513  Länstrafik - Buss 513           513   
2   Länstrafik - Buss X4   Länstrafik - Buss X4            X4   
3   Länstrafik - Buss X4   Länstrafik - Buss X4            X4   
4   Länstrafik - Buss 16   Länstrafik - Buss 16            16   

             catOutL  
0  Länstrafik - Buss  
1  Länstrafik - Buss  
2  Länstrafik - Buss  
3  Länstrafik - Buss  
4  Länstrafik - Buss  


In [5]:
from datetime import datetime

# Datetime column for departure time
df_combined["departure_time"] = pd.to_datetime(
    df_combined["date"] + " " + df_combined["time"]
)

# Calculate minutes to departure
current_time = datetime.now()
df_combined["minutes_to_departure"] = (
    df_combined["departure_time"] - current_time
).dt.total_seconds() / 60

print(
    df_combined[
        [
            "time",
            "date",
            "direction",
            "name",
            "displayNumber",
            "catOutL",
            "minutes_to_departure",
        ]
    ].head()
)

       time        date                       direction  \
0  08:46:00  2025-01-27               Göteborg Svingeln   
1  08:46:00  2025-01-27                  Göteborg Heden   
2  08:47:00  2025-01-27  Mölnlycke station (Härryda kn)   
3  08:47:00  2025-01-27             Kungälv resecentrum   
4  08:47:00  2025-01-27        Fyrktorget (Göteborg kn)   

                    name                   name displayNumber  \
0   Länstrafik - Buss X3   Länstrafik - Buss X3            X3   
1  Länstrafik - Buss 513  Länstrafik - Buss 513           513   
2   Länstrafik - Buss X4   Länstrafik - Buss X4            X4   
3   Länstrafik - Buss X4   Länstrafik - Buss X4            X4   
4   Länstrafik - Buss 16   Länstrafik - Buss 16            16   

             catOutL  minutes_to_departure  
0  Länstrafik - Buss             -0.639145  
1  Länstrafik - Buss             -0.639145  
2  Länstrafik - Buss              0.360855  
3  Länstrafik - Buss              0.360855  
4  Länstrafik - Buss         

In [6]:
# Filter departures within the next 60 minutes
df_next_hour = df_combined[df_combined["minutes_to_departure"] <= 60]

print(
    df_next_hour[
        [
            "time",
            "direction",
            "name",
            "displayNumber",
            "catOutL",
            "minutes_to_departure",
        ]
    ]
)

         time                           direction                    name  \
0    08:46:00                   Göteborg Svingeln    Länstrafik - Buss X3   
1    08:46:00                      Göteborg Heden   Länstrafik - Buss 513   
2    08:47:00      Mölnlycke station (Härryda kn)    Länstrafik - Buss X4   
3    08:47:00                 Kungälv resecentrum    Länstrafik - Buss X4   
4    08:47:00            Fyrktorget (Göteborg kn)    Länstrafik - Buss 16   
..        ...                                 ...                     ...   
189  09:45:00                  Kungsbacka station   Länstrafik - Tåg 3039   
190  09:45:00                      Göteborg Heden   Länstrafik - Buss 513   
191  09:46:00                 Kungälv resecentrum    Länstrafik - Buss X4   
192  09:46:00            Göteborg Östra sjukhuset    Länstrafik - Buss 17   
193  09:46:00  Kortedala Aprilgatan (Göteborg kn)  Länstrafik - Spårväg 6   

                       name displayNumber               catOutL  \
0      L

In [7]:
# Count transport types in the filtered data
print(df_next_hour["catOutL"].value_counts())

# Count unique lines in the filtered data
print(df_next_hour["displayNumber"].value_counts())

catOutL
Länstrafik - Buss       156
Länstrafik - Spårväg     15
Länstrafik - Tåg         10
Expressbuss               4
Regional Tåg              4
Flygtransfer - Buss       3
Snabbtåg                  2
Name: count, dtype: int64
displayNumber
X4       41
16       20
17       14
6        14
21       13
X3       12
X1       12
SVART    11
513       8
510       6
173       6
SNU       4
.         3
503       3
59        2
841       2
16X       1
1071      1
3738      1
3189      1
611       1
600       1
OEXP      1
487       1
3636      1
428       1
3037      1
356       1
20137     1
3536      1
7337      1
170       1
3187      1
613       1
3236      1
3634      1
TEXP      1
9         1
3039      1
Name: count, dtype: int64
